In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os
import glob
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd
import wfdb
import statsmodels.api as sm
import matplotlib.image as mpimg
import tkinter as tk
#from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2TkAgg)
from matplotlib.figure import Figure
from IPython.display import display, HTML
import csv
from collections import defaultdict
from sklearn.metrics import r2_score
import urllib
import ssl
import datetime

In [3]:
df = pd.read_csv('RecordsAnno.csv',sep=',')
df.head()

,RecordNum,PatientId,ICUStayId,HadmId,ARDS,PatternType,Age,Gender,LastStay,Mortality,Ventilator,Duration,SamplingRate,DateTimeStamp,ParametersAvailable,Category,RDSegs,DTW_RDSegs
0,3544749,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.916667,1,2183-04-28 17:47:59.486,"HR,ABPSys,ABPDias,ABPMean,PAPSys,PAPDias,PAPMe...",NaN,NaN,NaN
1,3524877,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1,2172-10-16 12:22:07.132,"HR,PULSE,RESP,SpO2,NBPSys,NBPDias,NBPMean",NaN,NaN,NaN
2,3713820,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.450000,1,2116-12-24 12:35:06.147,"HR,CVP,PULSE,RESP,SpO2,NBPSys,NBPDias,NBPMean",NaN,NaN,NaN
3,3737936,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.866667,1,2116-12-25 13:11:06.147,"HR,CVP,PULSE,RESP,SpO2,NBPSys,NBPDias,NBPMean",NaN,NaN,NaN
4,3238451,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.642778,60,2191-01-10 02:21:17.000,"HR,PULSE,NBPSys,NBPDias,NBPMean,RESP,SpO2,PVCR...",NaN,NaN,NaN


In [19]:
N = len(df)
myset = set()
for i in range(N):
    lst = df.ParametersAvailable[i].split(',')
    subset = set(lst)
    myset = myset.union(subset)
print(myset)

{'', 'ABPSYS', 'ST_I', 'T1', 'dSpO2', 'ST_II', 'P1MEAN', 'P4', 'ABPDias', 'UAPSys', 'TEMP', 'CVP_3/3', 'SpO2l', 'PVCRateperMinute', 'CI', 'ROnTCount', 'PacerNotPaceCount', 'STMCL', 'ABP', 'P1DIAS', '[62228]', 'Tblood', 'Tnaso', 'Tesoph', 'SVCount', 'P1Mean', 'BLOODT', 'SVPBRunCount', 'PRateMin', 'PacedBeatPercent', 'PVCRunCount', 'NBP', 'TempEsoph', 'ST_III', 'PAPMEAN', 'TempSkin', 'ETCO2', 'PAPSys', 'FAP_1/3', 'PAPSYS', 'BAPSYS', '[62229]', 'BigeminyPercent', 'IC2', 'UAPMean', 'UAPDias', 'PacedRunCount', 'EtCO2', 'DualPacedBeatCount', 'ST-aVF', 'P1Sys', 'Trect', 'ST-III', 'ST_V6', 'STV', 'STAVR', 'ST_V2', 'LAP', '%SpO2', 'TempNasal', 'SpO2R', 'PlsNBP', 'AoSYS', 'PAPDias', 'IC1', '[62130]_2/3', 'ICP', 'UVPDias', 'ST-aVR', 'BAP_1/3', 'FAP_2/3', 'PoorSignal', 'UAPMEAN', 'ARTDias', 'STAVF', 'ST-V', 'AOBP', 'ST_AVL', 'P1Dias', 'SVPBCount', 'BAP', 'UAPDIAS', 'ART', 'ST-MCL', 'ST_AVF', 'ST_V4', 'PAWP', 'P4DIAS', 'SpO2_AP', 'Tart', 'HRVariability', 'ABPMean', 'UVP', 'PacerNotCaptureCount', 'P

In [20]:
mylist = list(myset)
print(mylist)

['', 'ABPSYS', 'ST_I', 'T1', 'dSpO2', 'ST_II', 'P1MEAN', 'P4', 'ABPDias', 'UAPSys', 'TEMP', 'CVP_3/3', 'SpO2l', 'PVCRateperMinute', 'CI', 'ROnTCount', 'PacerNotPaceCount', 'STMCL', 'ABP', 'P1DIAS', '[62228]', 'Tblood', 'Tnaso', 'Tesoph', 'SVCount', 'P1Mean', 'BLOODT', 'SVPBRunCount', 'PRateMin', 'PacedBeatPercent', 'PVCRunCount', 'NBP', 'TempEsoph', 'ST_III', 'PAPMEAN', 'TempSkin', 'ETCO2', 'PAPSys', 'FAP_1/3', 'PAPSYS', 'BAPSYS', '[62229]', 'BigeminyPercent', 'IC2', 'UAPMean', 'UAPDias', 'PacedRunCount', 'EtCO2', 'DualPacedBeatCount', 'ST-aVF', 'P1Sys', 'Trect', 'ST-III', 'ST_V6', 'STV', 'STAVR', 'ST_V2', 'LAP', '%SpO2', 'TempNasal', 'SpO2R', 'PlsNBP', 'AoSYS', 'PAPDias', 'IC1', '[62130]_2/3', 'ICP', 'UVPDias', 'ST-aVR', 'BAP_1/3', 'FAP_2/3', 'PoorSignal', 'UAPMEAN', 'ARTDias', 'STAVF', 'ST-V', 'AOBP', 'ST_AVL', 'P1Dias', 'SVPBCount', 'BAP', 'UAPDIAS', 'ART', 'ST-MCL', 'ST_AVF', 'ST_V4', 'PAWP', 'P4DIAS', 'SpO2_AP', 'Tart', 'HRVariability', 'ABPMean', 'UVP', 'PacerNotCaptureCount', 'P

In [24]:
mydf = pd.DataFrame(columns=['ParameterName','ParameterDescription'])
mydf.ParameterName = mylist
display(mydf)

,ParameterName,ParameterDescription
0,,NaN
1,ABPSYS,NaN
2,ST_I,NaN
3,T1,NaN
4,dSpO2,NaN
5,ST_II,NaN
6,P1MEAN,NaN
7,P4,NaN
8,ABPDias,NaN
9,UAPSys,NaN


In [25]:
mydf.to_csv('ParameterList.csv',index=False)